In [ ]:
import os 
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import sys
sys.path.append("..")

from make_dir import mkdir
from load_yaml import get_yaml

import models.net as solutions
import equations.poisson_eqn as equation

from dataset import Sampler
import solver as solver 

import matplotlib.pyplot as plt
import time



In [ ]:
# load config
current_path = os.path.abspath(".")
yaml_path = os.path.join(current_path, "config.yaml")
Config = get_yaml(yaml_path)


In [ ]:
space_dimension = Config["physical_config"]["space_dimension"]
d_in = space_dimension
layers = Config["model_config"]["units"]

# build neural networks for f
Model = "solutions.Model_" + \
    "{}".format(Config["model_config"]["neural_network_type"])
Model = eval(Model)

# approx theta and phi with neural networks
model = Model(input_size = d_in, layers = layers, output_size = 1)

device_ids = Config["model_config"]["device_ids"]
device = torch.device("cuda:{:d}".format(device_ids[0]) if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:    
    model = nn.DataParallel(model, device_ids = device_ids)
    
model.to(device)



In [ ]:
# number of paramerters
param_num = sum(neural.numel() for neural in model.parameters())
print("Number of paramerters for networks is: {:6d}. ".format(param_num))


In [ ]:
solutions.Xavier_initi(model)


In [ ]:
# Set optimizer and learning rate decay
optimizer = optim.Adam([
    {'params': model.parameters()},
],  lr=Config["model_config"]["lr"])

scheduler = lr_scheduler.StepLR(
    optimizer, Config["model_config"]["stage_num"], Config["model_config"]["decay_rate"])


In [ ]:
Sol = model
eqn = equation.Poisson(config = Config, sol = Sol)

In [ ]:
Iter = Config["model_config"]["iterations"] 
regularizers = Config["model_config"]["regularizers"]

loss_record, error_record = np.array([[]]).T, np.array([[]]*1).T

mkdir(file_dir = "./model_saved")
mkdir(file_dir = "./record")
mkdir(file_dir = "./figure")

time_start = time.time()
print('Begin training.')
print('')
for it in range(Iter):
    
    sampler = Sampler(Config)
    trainloader = [sampler.interior(),]
        
    risk, error = solver.train_step(sol = Sol,
                                    trainloader = trainloader, 
                                    equation = eqn,  
                                    regularizers = regularizers,
                                    optimizer = optimizer, 
                                    scheduler = scheduler)
    
    loss = risk["total_loss"]
    res_eqn = risk["eqn"]
    res_bc = risk["bc"]
    error_u = error["u"]

    error = np.array([error_u], dtype=float).reshape(1, -1)
    loss_record = np.concatenate((loss_record, loss*np.ones((1, 1))), axis=0)
    error_record = np.concatenate((error_record, error), axis=0)

    lr = optimizer.state_dict()['param_groups'][0]['lr']
    
    if it % 100 == 0:
    
        print("[Iter: {:6d}/{:6d} - lr: {:.2e} and Loss: {:.2e}]".format(it + 1, Iter, lr, loss))
        print("[Error for u: {:.2e}]".format(float(error[:, 0])))
        print("[Eqn: {:.2e}, Boundary: {:.2e}]".format(res_eqn, res_bc))

np.savez("./record/result.npz",
         loss=loss_record,
         error=error_record[:, 0])

solutions.save_param(model, path = './model_saved/model_params.pkl')

print("")
print("Finished training.")
time_end = time.time()
print("Total time is: {:.2e}".format(time_end - time_start), "seconds")
